In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для тайм-менеджмента
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from tqdm import tqdm
from time import sleep
from webdriver_manager.firefox import GeckoDriverManager
from datetime import datetime
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.by import By
from sys import platform
import nltk
from selenium.webdriver.common.keys import Keys
from random import SystemRandom
from selenium import webdriver
from nltk.tokenize import TweetTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from string import punctuation
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/victor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [127]:
df = pd.DataFrame()

In [128]:
def sleep_rand(min_sec, max_sec=None, rand_gen=SystemRandom()):
    
    """
        Задержка на произвольное время от min секунд до max секунд.
        Точность - десятые доли секунды.

        :param rand_gen: генератор случайных чисел (объект random.SystemRandom())
        :param min_sec: минимальное число секунд
        :param max_sec: максимальное число секунд.
    """

    if not max_sec:
        max_sec = min_sec + 1
    if min_sec > max_sec:
        print('Неверный интервал!')
    else:
        sleep(rand_gen.randint(min_sec, max_sec - 1) + rand_gen.randint(0, 10) / 10)

        
def init(link, options = None):
    """
    link - ссылка на страницу
    options - [list of str] - настройки драйвера 
    """
    profile = webdriver.FirefoxProfile()
    profile.set_preference("network.proxy.type", 1)
    profile.set_preference("network.proxy.socks", '127.0.0.1')
    #profile.set_preference("network.proxy.socks_port", tor_port)  # Порт, который слушает Tor
    profile.set_preference("network.proxy.socks_remote_dns", False)
    profile.set_preference( "intl.accept_languages", "ru-ru" )
    profile.update_preferences()
    if options:
        opt = FirefoxOptions()
        for option in options:
            opt.add_argument(option)
        driver = webdriver.Firefox(firefox_profile=profile, firefox_options = opt, executable_path=GeckoDriverManager().install())
        driver.get(page_link)
        sleep_rand(2)
        return driver
    driver = webdriver.Firefox(firefox_profile=profile, executable_path=GeckoDriverManager().install())
    driver.get(page_link)
    sleep_rand(2)
    return driver
    
def auth(driver: webdriver.Firefox, 
        login: str, password: str):
    
    
    

    # Задержки - для имитации работы человека в браузере
    sleep_rand(1)

    # Находим окошко введения логина, вводим логин
    login_bar_xpath = '/html/body/div[1]/section/main/article/div[2]/div[1]/div/form/div/div[1]/div/label/input'
    login_bar = driver.find_elements_by_xpath(login_bar_xpath)[0]
    sleep_rand(1)

    login_bar.clear()
    login_bar.click()
    sleep_rand(1)

    login_bar.send_keys(login)
    sleep_rand(1)

    # Аналогично вводим пароль
    pass_bar_xpath = '/html/body/div[1]/section/main/article/div[2]/div[1]/div/form/div/div[2]/div/label/input'
    pass_bar = driver.find_elements_by_xpath(pass_bar_xpath)[0]
    sleep_rand(1)

    pass_bar.clear()
    pass_bar.click()
    sleep_rand(1)

    pass_bar.send_keys(password)
    sleep_rand(1)

    pass_bar.send_keys(Keys.RETURN) # имитируем нажатие клавиши Enter
    sleep_rand(3)
    #кнопка сохранить аккаунт, мы не будем сохранять. может не выскочить
    save_acc_xpath = '/html/body/div[1]/section/main/div/div/div/div/button'
    save_acc_button = driver.find_elements_by_xpath(save_acc_xpath)[0]
    try:
        save_acc_button.click()
        print('1 кнопка нажата')
    except:
        print('1 кнопки не было')
    sleep_rand(3)
    
    alert_button = driver.find_elements_by_xpath("//button[contains(text(), 'Не сейчас')]")[0]
    try:
        alert_button.click()
        print('2 кнопка нажата')
    except:
        print('2 кнопки не было')
        
    
    #нужно кликнуть на поиск, изначально инст считает что кнопку поиска что-то загораживает
    caph = input()

def search(request):
    ctrl_button_xpath = '/html/body/div[1]/section/nav/div[2]/div/div/div[2]'
    ctrl_button = driver.find_element_by_xpath(ctrl_button_xpath)
    search_bar_xpath = '/html/body/div[1]/section/nav/div[2]/div/div/div[2]/input'
    search_bar = driver.find_element_by_xpath(search_bar_xpath)
    try:
        search_bar.clear()
    except:
        pass
    search_bar.click()
    sleep_rand(2)
    search_bar.send_keys(request)
    
    sleep_rand(3)
    selen_page = driver.page_source
    soup = BeautifulSoup(selen_page,'html.parser')
    
    
    res = soup.findAll('a', attrs = {'class':"yCE8d"})
    
    #no hashtags
    hs = []
    for num, acc in enumerate(res):
        if acc.text.startswith('#'):
            hs.append(num)
    for i in sorted(hs, reverse=True):
         del res[i]
            
    acc_name = []
    acc_link = []
    for acc in res:
        acc_name.append(acc.text)
        acc_link.append(page_link+acc['href'])
    
    global df
    if df.empty:
        df = pd.DataFrame({ 'name': acc_name, 'link':acc_link })
    else:
        df = df.append(pd.DataFrame({ 'name': acc_name, 'link':acc_link}), ignore_index = True)
    df = df.drop_duplicates()

In [129]:
login = 'pathtest2020'
password = 'dataparsing'
page_link = 'https://www.instagram.com'


In [130]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
driver.get(page_link)

[WDM] - Driver [/home/victor/.wdm/drivers/geckodriver/linux64/v0.28.0/geckodriver] found in cache


In [131]:
auth(driver, login, password)

1 кнопка нажата
2 кнопка нажата
1


In [142]:
df

,name,link
0,babyshop_ulluayaКорейская Детская Одежда,https://www.instagram.com/babyshop_ulluaya/
1,umulimagomedovодежда H&M ....KIABI,https://www.instagram.com/umulimagomedov/
2,achxoy_butikОДЕЖДА И ОБУВЬ,https://www.instagram.com/achxoy_butik/
3,borz_homeBORZ HOME | ОДЕЖДА ДЛЯ ДОМА,https://www.instagram.com/borz_home/
4,zara.mutsalaulодежда,https://www.instagram.com/zara.mutsalaul/
...,...,...
83,remont_bitovoy_tehniki.kgРемонт бытовой техники!,https://www.instagram.com/remont_bitovoy_tehni...
84,nata0308Материнство Дети Обзоры Ремонт,https://www.instagram.com/nata0308/
85,gsmservicekzРЕМОНТ ТЕЛЕФОНОВ И НОУТБУКОВ,https://www.instagram.com/gsmservicekz/
86,marina_at_homeХрущёвка•Ремонт•Интерьер• DIY,https://www.instagram.com/marina_at_home/


In [132]:
search('одежда')
search('ремонт')

In [138]:
driver.get(df['link'][1])

In [146]:
selen_page = driver.page_source
soup = BeautifulSoup(selen_page,'html.parser')

In [147]:
soup.findAll('span', attrs = {'span':"-g47SY "})

[]

In [17]:
driver.find_elements_by_xpath('/html/body/div[1]/section/nav/div[2]/div/div/div[1]/a/div/div/img')[0].click()

In [29]:
driver.find_elements_by_xpath('/html/body/div[1]/section/nav/div[2]/div/div/div[2]/input')[0].click()

In [31]:
suka = '/html/body/div[1]/section/nav/div[2]/div/div/div[2]'
suka1 = driver.find_element_by_xpath(suka)
search_bar_xpath = '/html/body/div[1]/section/nav/div[2]/div/div/div[2]/input'
search_bar = driver.find_element_by_xpath(search_bar_xpath)
search_bar.clear()
search_bar.click()
sleep_rand(2)
search_bar.send_keys('одежда1')

In [32]:
selen_page = driver.page_source
soup = BeautifulSoup(selen_page,'html.parser')

In [33]:
soup

<html class="js logged-in client-root js-focus-visible sDN5V" lang="ru"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>Instagram</title>
<meta content="noimageindex, noarchive" name="robots"/>
<meta content="default" name="apple-mobile-web-app-status-bar-style"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="#ffffff" name="theme-color"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover" id="viewport" name="viewport"/>
<link href="/data/manifest.json" rel="manifest"/>
<link as="style" crossorigin="anonymous" href="/static/bundles/es6/ConsumerUICommons.css/6a9060457779.css" rel="preload" type="text/css"/>
<link as="style" crossorigin="anonymous" href="/static/bundles/es6/Consumer.css/76626a1f11f7.css" rel="preload" type="text/css"/>
<link as="style" crossorigin="anonymous" href="/static/bundles/es6/FeedPageContainer.css/1b39124c7396.css" rel="preload" type="text/css"

In [42]:
res = soup.findAll('a', attrs = {'class':"yCE8d"})

In [57]:
res[3]['href']

'/sadovod_merkur_17_97/'

In [46]:
for num, acc in enumerate(res):
    if acc.text.startswith('#'):
        res.pop(num)
        print(num)

In [38]:
a = ['a', 'b']
for num, b in enumerate(a):
    print(num,b)

0 a
1 b


In [2]:
#обозначили сайт, проверили, чтобы 200 ок
page_link = 'https://www.instagram.com/?hl=ru'
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [4]:
#вызов нужного сайта
driver.get(page_link)
time.sleep(6)
ok_button = driver.find_elements_by_xpath("//div[contains(text(), 'OK')]")
try:
    ok_button[0].click()
except:
    pass

In [20]:
login_bar.clear()
login_bar.click()
sleep_rand(1)

login_bar.send_keys(login)
sleep_rand(1)

In [16]:
login_bar_xpath = '/html/body/div[1]/section/main/article/div[2]/div[1]/div/form/div/div[1]/div/label/input'
pass_bar_xpath = '/html/body/div[1]/section/main/article/div[2]/div[1]/div/form/div/div[2]/div/label/input'
login_bar = driver.find_elements_by_xpath(login_bar_xpath)[0]
pass_bar = driver.find_elements_by_xpath(pass_bar_xpath)[0]


In [7]:
#Количество скроллов вниз, чем больше, тем больше получим записей. Но дольше будет работать
times_scroll = 50

In [15]:
#Цикл для повторения прокрутки
for i in tqdm(range(times_scroll)):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    if i%5==0: #Здесь каждую пятую прокрутку цикл засыпает на 1 секунду, чтобы страница прогрузилась
               #Если убрать эту строчку, может не дорабатывать до конца и прогружать меньше
        time.sleep(1)

100%|██████████| 50/50 [00:11<00:00,  4.53it/s]


In [9]:
#Костыли для открытия комментов
buttons = driver.find_elements_by_xpath("//div[contains(text(), 'Ещё')]")
for i in tqdm(range(len(buttons))):
    try: 
        buttons[i].click()
    except:
        continue

#Костыли, созданные, чтобы открыть все длинные посты. К сожалению к двум циклам пришлось прибегнуть, так как
#часть кнопок не нажимаются с первого раза из-за того, что сайт недогрузился или не доскролился до нужной кнопки            
for i in range(2):       
    should_restart = True
    while should_restart:
        should_restart = False
        buttons = driver.find_elements_by_xpath("//span[contains(text(), 'Показать ещё')]")
        for i in range(len(buttons)):
            try: 
                buttons[i].click()
            except:
                should_restart = True
                break

buttons = driver.find_elements_by_xpath("//span[contains(text(), 'ответ')]")
for i in tqdm(range(len(buttons))):
    try: 
        buttons[i].click()
    except:
        continue
buttons = driver.find_elements_by_xpath("//span[contains(text(), 'ответов')]")
for i in tqdm(range(len(buttons))):
    try: 
        buttons[i].click()
    except:
        continue

100%|██████████| 23/23 [00:11<00:00,  1.98it/s]


In [16]:
# Чтение HTML Страницы
selen_page = driver.page_source
soup = BeautifulSoup(selen_page,'html.parser')

In [17]:
posts = (soup.findAll('div', attrs = {'class':"kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q"}))
b = soup.findAll('span', attrs = {'class':"j1lvzwm4 stjgntxs ni8dbmo4 q9uorilb gpro0wi8"})
#текст постов и время размещения *не спрашивайте почему b*

In [18]:
b

[<span aria-label="22 ч." aria-labelledby="jsc_c_b" class="j1lvzwm4 stjgntxs ni8dbmo4 q9uorilb gpro0wi8"><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4">22 ч.<b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4" style="display: none;">=</b><b class="b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a26

In [19]:
#from datetime import timedelta
import datetime
import time

In [20]:
(datetime.datetime.now() - datetime.timedelta (days = 1)).day

11

In [21]:
#Загрузка сегодняшней даты в нужном формате
day_td = str(datetime.datetime.now().day)
day_ytd = str((datetime.datetime.now() - datetime.timedelta (days = 1)).day)
month_td = list(MonthDict.keys())[list(MonthDict.values()).index(datetime.datetime.now().month)]
year_td = str(datetime.datetime.now().year)
date_td = day_td+month_td+year_td
date_ytd = day_ytd+month_td+year_td
#Куча ИФ циклов для чистки и преобразования данных о времени
parsed_ls = []
#Первый цикл, для создания листа, где не будет лишних знаков
for i in tqdm(range(len(b))):
    parsed = (b[i].text.replace('=','').replace('·','').replace('\xa0', ''))
    if b[i].text != '':
        parsed_ls.append(parsed)
#Выкидывание ненужной информации и замена часов, минут и дней на дату
for i in tqdm(range(len(parsed_ls))):
    parsed_ls[i] = parsed_ls[i].replace(' ', '')
    parsed_ls[i] = parsed_ls[i].replace('г.Москва', '')
    parsed_ls[i] = parsed_ls[i].replace('г.', '')
    if 'ч.' in parsed_ls[i]:
        parsed_ls[i] = date_td
    if 'мин.' in parsed_ls[i]:
        parsed_ls[i] = date_td
    if 'Вчера' in parsed_ls[i]:
        parsed_ls[i] = date_ytd
#Перевод дат в формат д/м/г
for i in tqdm(range(len(parsed_ls))):
    if (parsed_ls[i][-3]==':')==True:
        parsed_ls[i]= parsed_ls[i][:-6]+year_td
    if parsed_ls[i][-4:-2]!='20':
        parsed_ls[i] = parsed_ls[i]+parsed_ls[i-1][-4:]
    if parsed_ls[i][:2].isnumeric():
        month = str(MonthDict[parsed_ls[i][2:-4]])
        date = (parsed_ls[i][:2])
        year = (parsed_ls[i][-4:])
        parsed_ls[i] = date+'/'+month+'/'+year
    if parsed_ls[i][:2].isnumeric()==False:
        month = str(MonthDict[parsed_ls[i][1:-4]])
        date = (parsed_ls[i][:1])
        year = (parsed_ls[i][-4:])
        parsed_ls[i] = date+'/'+month+'/'+year

100%|██████████| 45/45 [00:00<00:00, 178396.67it/s]


In [22]:
#перевод дат в формат datetime, перевод листа с хтмл кодом в текстовый лист
dt_ls = []
for date in (parsed_ls):
    dt_ls.append(datetime.datetime.strptime(date, '%d/%m/%Y'))
for i in range(len(posts)):
    posts[i] = posts[i].text

In [23]:
#Создание датафрейма
Dict = {'time': dt_ls[:len(posts)], 'post': posts}
df = pd.DataFrame.from_dict(Dict)

In [24]:
new_cl = 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql gk29lw5a a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d9wwppkn fe6kdd0r mau55g9w c8b282yb hrzyx87i jq4qci2q a3bd9o3v knj5qynh m9osqain'

In [25]:
#считывание комментариев и количества комментариев под каждым постом
comms = (soup.findAll('div', attrs = {'class':"tw6a2znq sj5x9vvc d1544ag0 cxgpxx05"}))
#comm_num_cl = 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql gk29lw5a a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d9wwppkn fe6kdd0r mau55g9w c8b282yb hrzyx87i jq4qci2q a3bd9o3v knj5qynh m9osqain'
comm_num = (soup.findAll('span', attrs = {'class':new_cl}))

In [26]:
#выкидываем кол-во репостов
comm_num_ls = []
for i in tqdm(range(len(comm_num))):
    if 'Комментарии' in comm_num[i].text:
        comm_num_ls.append(int(comm_num[i].text.replace('Комментарии: ', '')))
#почти всегда получается так, что посты без комментов технически сьедают комменты,
#а нижние посты остаются без них. Добьем лист нулями. Общая картина текстов не теряется
if len(comm_num_ls)<len(df):
    for i in range(len(df)-len(comm_num_ls)):
        comm_num_ls.append(0)

100%|██████████| 54/54 [00:00<00:00, 56397.51it/s]


In [27]:
#Добавляем колонку с кол-вом комментов
df['number_comments'] = np.array(comm_num_ls[:len(df)])

In [28]:
#токенизируем комменты, чтобы выкинуть мусор и убрать обращение по имени
#шумовые слова выкидываю сразу, чтобы не засорять датасет
tw = TweetTokenizer()
det = TreebankWordDetokenizer()
noise = stopwords.words('russian') + list(punctuation)
upnoise = [letter.upper() for letter in noise]
sum_noise = noise+upnoise+['.','»','«']
for i in tqdm(range(len(comms))):
    comms[i] = comms[i].text
    tokenized_example = (tw.tokenize(comms[i])[2:])
    #filtered_example = [word for word in tokenized_example if not word in noise]
    comms[i] = det.detokenize(tokenized_example)

100%|██████████| 65/65 [00:00<00:00, 7031.43it/s]


In [29]:
#разбиваем и клеим так, чтобы они были одним предложением. А также чтобы у каждого поста нужное кол-во
comms_ls = []
j = 0 
for i in tqdm(range(len(df))):
    comms_ls.append(comms[j:j+df.iloc[i,2]])
    j+=df.iloc[i,2]

#for i in range(len(comms_ls)):
 #   sentnc = ''
 #   for j in range(len(comms_ls[i])):
 #       sentnc += comms_ls[i][j]
  #      sentnc += ' '
 #   comms_ls[i] = sentnc
df['comment'] = comms_ls

100%|██████████| 38/38 [00:00<00:00, 38102.69it/s]


In [30]:
df

,time,post,number_comments,comment
0,2020-11-12,Вопрос к знатокам: Задача купить максимально д...,6,"[из немногих постов о том о чем нужно! ..., !,..."
1,2020-11-11,Офисные истории агентств недвижимости:,87,[она почему кв-ру сдает за деньги??? И ведь ск...
2,2020-11-10,Я просто повеселится.,8,[]
3,2020-11-10,"Коллеги добрый день. Каким образом, что/какой...",7,[]
4,2020-11-12,https://m.facebook.com/story.php?story_fbid=34...,229,[]
5,2020-10-20,"Давно не было от меня вестей. Все потому, что,...",24,[]
6,2020-11-09,#Агентство_недвижимости #Top #TheDom,9,[]
7,2020-11-10,Гениальный ремонт: хрущёвка серии I-464А в Мос...,25,[]
8,2020-11-10,"Мне вроде кажется, что радоваться тут нечему.",15,[]
9,2020-11-08,Армагедон продолжается.,110,[]


In [102]:
df.to_csv('fb.csv')

In [104]:
df.to_excel('exfb.xlsx')